<a href="https://colab.research.google.com/github/scottthomaswx/ProjectsInProgress/blob/main/FVCOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Needed Packages

In [169]:
# @title
!pip install datetime --quiet
!pip install numpy --quiet
!pip install matplotlib  -quiet
!pip install cartopy --quiet
!pip install xarray[complete] --quiet
!pip install metpy --quiet
!pip install pandas --quiet
!pip install cfgrib
!apt-get install -y libeccodes0
!pip install PIL


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libeccodes0 is already the newest version (2.24.2-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [171]:
import requests
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import cartopy.crs as ccrs
import xarray as xr
from metpy import units
import cartopy.feature as cfeature
import pandas as pd
from metpy.plots import USCOUNTIES
import os
from PIL import Image
!rm -rf img_files
!rm -rf fvcom_data
!mkdir fvcom_data
!mkdir img_files

mkdir: cannot create directory ‘fvcom_data’: File exists
mkdir: cannot create directory ‘img_files’: File exists


In [96]:
year = datetime.now().year
month = datetime.now().month
day = datetime.now().day
hour = datetime.now().hour
fhour_list = np.arange(0,72,1)
q=0
if hour < 16:
  run_hour = '00'
else:
  run_hour = '12'

In [115]:
url1 = f'https://noaa-nos-ofs-pds.s3.amazonaws.com/lmhofs/netcdf/{year}/{month:02}/{day:02}/lmhofs.t{run_hour}z.{year}{month:02}{day:02}.stations.forecast.nc'
r1 = requests.get(url1)
file1= f'/content/fvcom_data/{year}{month}{day}.t{run_hour}z.fvcom.station.nc'
with open(file1, 'wb') as f1:
      f1.write(r1.content)

In [111]:
fvcom_stations = xr.open_mfdataset('/content/fvcom_data/*station*nc', engine="netcdf4", combine='nested', concat_dim='time', decode_times=False)

In [112]:
BIV = fvcom_stations.isel(station=np.where(fvcom_stations.name_station.values == 9087031)[0])
LDM = fvcom_stations.isel(station=np.where(fvcom_stations.name_station.values == 9087032)[0])

In [119]:
while q < len(fhour_list):
  url1 = f'https://noaa-nos-ofs-pds.s3.amazonaws.com/lmhofs/netcdf/{year}/{month:02}/{day:02}/lmhofs.t{run_hour}z.{year}{month:02}{day:02}.regulargrid.f{fhour_list[q]:03}.nc'
  r1 = requests.get(url1)
  file1= f'/content/fvcom_data/{year}{month}{day}.t{run_hour}z.fvcom.f{fhour_list[q]:03}.nc'
  with open(file1, 'wb') as f1:
        f1.write(r1.content)
  q+=1

In [126]:
fvcom_datagrid = xr.open_mfdataset('./fvcom_data/*f*nc', engine="netcdf4",
                        concat_dim="time", combine='nested')
fvcom_datagrid = fvcom_datagrid.isel(Depth=0)
water_level = fvcom_datagrid['zetatomllw'].metpy.convert_units('ft')

In [ ]:
l=0
colors = ['#deebf7', '#deebf7','#deebf7', '#9dc3e6', '#9dc3e6', '#0070c0',
          '#0070c0', '#060082','#060082', '#81e78d', '#81e78d', '#00b050',
          '#00b050', '#ffff00','#ff9900', '#ff0000', '#ffb6c1', '#7030a0']
cmap_plot = matplotlib.colors.LinearSegmentedColormap.from_list("",colors)
levels = [-2.0,-1.5,-1.0,-0.5,0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
while l < len(water_level['time']):
  fig = plt.figure(figsize=(14,8), facecolor='white', edgecolor = 'black', linewidth = 6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent([-89,-85,41.5, 46])
  waterht = ax.contourf(water_level.Longitude, water_level.Latitude, water_level.isel(time=l), levels = levels, cmap = cmap_plot)
  waterhtcon = ax.contour(water_level.Longitude, water_level.Latitude, water_level.isel(time=l), colors = 'white', linewidths = 1,levels = levels)
  ax.clabel(waterhtcon, fontsize = 16, colors = 'white')
  cbar = plt.colorbar(waterht, shrink=0.9, orientation = 'vertical')
  cbar.set_label('Height Above Low Water Level (ft)', rotation=270)
  ax.coastlines(color='black',resolution='10m')
  ax.add_feature(cfeature.STATES.with_scale('50m'),edgecolor='darkgray')
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='lightgray', alpha = 0.8, zorder = 0)
  plt.title(f'FVCOM Height Above Low Water FHR{l:02}', loc = 'left')
  plt.title(f'Run:{year}-{month}-{day} {run_hour}z', loc = 'right')
  plt.figtext(0.53, 0.07, f'Advisory: +5.0 Ft', ha='left', weight='bold', color = '#ff9900', fontsize=15)
  plt.figtext(0.36, 0.07, f'Warning: +5.5 Ft', ha='left', weight='bold', color = 'red', fontsize=15)

  plt.savefig(f'/content/img_files/fvcom_waterlevel_{year}_{month}_{day}_{run_hour}_FHR{l:02}.png', bbox_inches = 'tight')
  l+=1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-179-de16e834fe6e>", line 11, in <cell line: 0>
    waterht = ax.contourf(water_level.Longitude, water_level.Latitude, water_level.isel(time=l), levels = levels, cmap = cmap_plot)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cartopy/mpl/geoaxes.py", line 306, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cartopy/mpl/geoaxes.py", line 350, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cartopy/mpl/geoaxes.py", line 1646, in contourf
    result = super().contourf(*

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <function flush_figures at 0x799d5ecb82c0> (for post_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-179-de16e834fe6e>", line 11, in <cell line: 0>
    waterht = ax.contourf(water_level.Longitude, water_level.Latitude, water_level.isel(time=l), levels = levels, cmap = cmap_plot)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cartopy/mpl/geoaxes.py", line 306, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cartopy/mpl/geoaxes.py", line 350, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-package

In [173]:
def create_gif(image_folder, output_gif, duration=500):
    images = []
    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith((".jpg", ".jpeg", ".png", ".bmp")):
            filepath = os.path.join(image_folder, filename)
            images.append(Image.open(filepath))

    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        optimize=False,
        duration=duration,
        loop=0,
    )

if __name__ == "__main__":
    image_folder = "/content/img_files/"
    output_gif = "fvcom_waterlwd.gif"
    create_gif(image_folder, output_gif)